In [2]:
#Khai báo các thư viện để xử lý dữ liệu
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, max, min, sum, log, log10, log2
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, ChiSqSelector
from pyspark.ml.classification import LinearSVC, RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import sqlite3
import pandas as pd
import ldata

In [2]:
#Load dữ liệu arff sang pandas Dataframe
file_path= '/root/NSLKDDProject/nslkdd/KDDTrain+.arff'
df = ldata.load_arff_to_pandasDF(file_path)
#print(df)
# Khởi tạo Spark session
spark = SparkSession.builder.appName("NSLKDDApp").getOrCreate()
#Load DataFrame
spark_df = spark.createDataFrame(df)
#Show dữ liệu 
spark_df.show()
total_records = spark_df.count()
print(f"Tổng số lượng record: {total_records}")
#Đếm số lượng record của mỗi class
class_counts = spark_df.groupBy("class").count()
class_counts.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/05 15:24:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/05 15:24:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/09/05 15:24:54 WARN TaskSetManager: Stage 0 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


+--------+-------------+----------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|   service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|d

24/09/05 15:24:56 WARN TaskSetManager: Stage 1 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


Tổng số lượng record: 125973


24/09/05 15:24:57 WARN TaskSetManager: Stage 4 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


+-------+-----+
|  class|count|
+-------+-----+
| normal|67343|
|anomaly|58630|
+-------+-----+



In [3]:
# Mô tả dữ liệu (statstics cho mỗi cột)
# count: Kiểm tra giá trị không null
# mean: Giá trị trung bình
# stddev: Độ lệch chuẩn
# min: giá trị lớn nhất
# max: Giá trị nhỏ nhất
description = spark_df.describe()
description.show()
spark_df.printSchema()
column_types = spark_df.dtypes
# Print the data type    
type_counts = {}
for _, dtype in column_types:
    if dtype in type_counts:
        type_counts[dtype] += 1
    else:
        type_counts[dtype] = 1

# Đếm giá trị các trường dữ liệu
for dtype, count in type_counts.items():
    print(f"Data Type: {dtype}, Count: {count}")
print("")  
for column, dtype in column_types:
    if (dtype == 'string'):
        print(f"Column: {column}, Type: {dtype}")
 

24/09/05 15:25:11 WARN TaskSetManager: Stage 7 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


+-------+-----------------+-------------+-------+------+-----------------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+------------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+------------------+------------------+-------------------+-----------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|summary|         duration|protocol_type|service|  flag|        src_bytes|         dst_bytes|                land|      wrong_fragment|              urgent|        

In [4]:
#Xem xét một số trường dữ liệu 
#Ví dụ về protocol_type
protocol_counts = spark_df.groupBy("protocol_type").count()
protocol_counts.show()
service_counts = spark_df.groupBy("service").count()
service_counts.show()
flag_counts = spark_df.groupBy("flag").count()
flag_counts.show()
#service_percentage = service_counts.withColumn("Percentage", (col("service") / total_records) * 100)
#service_percentage.show()

#Kiểm tra giá trị null trên dữ liệu
null_counts = {}
for checkcol in spark_df.columns:
    null_count = spark_df.filter(spark_df[checkcol].isNull()).count()
    null_counts[checkcol] = null_count
print(null_counts)

24/09/05 15:25:22 WARN TaskSetManager: Stage 10 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|102689|
|          udp| 14993|
|         icmp|  8291|
+-------------+------+



24/09/05 15:25:22 WARN TaskSetManager: Stage 13 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


+--------+-----+
| service|count|
+--------+-----+
|  telnet| 2353|
|     ftp| 1754|
|    auth|  955|
|iso_tsap|  687|
|  systat|  477|
|    name|  451|
| sql_net|  245|
|   ntp_u|  168|
|     X11|   73|
|   pop_3|  264|
|    ldap|  410|
| discard|  538|
|  Z39_50|  862|
| daytime|  521|
|domain_u| 9043|
|   login|  429|
|    smtp| 7313|
|     mtp|  439|
|  domain|  569|
|    http|40338|
+--------+-----+
only showing top 20 rows



24/09/05 15:25:23 WARN TaskSetManager: Stage 16 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:24 WARN TaskSetManager: Stage 19 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


+------+-----+
|  flag|count|
+------+-----+
|RSTOS0|  103|
|    S3|   49|
|    SF|74945|
|    S0|34851|
|   OTH|   46|
|   REJ|11233|
|  RSTO| 1562|
|  RSTR| 2421|
|    SH|  271|
|    S2|  127|
|    S1|  365|
+------+-----+



24/09/05 15:25:25 WARN TaskSetManager: Stage 22 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:25 WARN TaskSetManager: Stage 25 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:26 WARN TaskSetManager: Stage 28 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:26 WARN TaskSetManager: Stage 31 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:27 WARN TaskSetManager: Stage 34 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:27 WARN TaskSetManager: Stage 37 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:28 WARN TaskSetManager: Stage 40 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.

{'duration': 0, 'protocol_type': 0, 'service': 0, 'flag': 0, 'src_bytes': 0, 'dst_bytes': 0, 'land': 0, 'wrong_fragment': 0, 'urgent': 0, 'hot': 0, 'num_failed_logins': 0, 'logged_in': 0, 'num_compromised': 0, 'root_shell': 0, 'su_attempted': 0, 'num_root': 0, 'num_file_creations': 0, 'num_shells': 0, 'num_access_files': 0, 'num_outbound_cmds': 0, 'is_host_login': 0, 'is_guest_login': 0, 'count': 0, 'srv_count': 0, 'serror_rate': 0, 'srv_serror_rate': 0, 'rerror_rate': 0, 'srv_rerror_rate': 0, 'same_srv_rate': 0, 'diff_srv_rate': 0, 'srv_diff_host_rate': 0, 'dst_host_count': 0, 'dst_host_srv_count': 0, 'dst_host_same_srv_rate': 0, 'dst_host_diff_srv_rate': 0, 'dst_host_same_src_port_rate': 0, 'dst_host_srv_diff_host_rate': 0, 'dst_host_serror_rate': 0, 'dst_host_srv_serror_rate': 0, 'dst_host_rerror_rate': 0, 'dst_host_srv_rerror_rate': 0, 'class': 0}


In [5]:
# THỰC HIỆN PREPROCESSING DATA BAO GỒM
#0. Drop cột
spark_df = spark_df.drop("num_outbound_cmds")
#1. One hot encoding data string dựa trên tần suất xuất hiện của value trong data
translist = ["protocol_type", "service", "flag"]
translist_temp = ["protocol_type_trans", "service_trans", "flag_trans"]
for i in range(len(translist)):
    indexer = StringIndexer(inputCol=translist[i], outputCol=translist_temp[i])
    spark_df = indexer.fit(spark_df).transform(spark_df)
    spark_df = spark_df.drop(translist[i]).withColumnRenamed(translist_temp[i], translist[i])

#2. Chuyển đổi các cột string cast về double
castlistvalue = ["land", "logged_in", "is_host_login", "is_guest_login"]
for i in range (len(castlistvalue)):
    spark_df = spark_df.withColumn(castlistvalue[i], col(castlistvalue[i]).cast(DoubleType()))

#3 Logarithmic scaling method dành cho các trường dữ liệu có độ chênh lệch lớn giữa max và min value
#loglist = ["duration", "src_bytes", "dst_bytes", "num_compromised","num_root"]
#loglist_temp = ["duration_trans", "src_bytes_trans", "dst_bytes_trans", "num_compromised_trans","num_root_trans"]
loglist = ["duration", "src_bytes", "dst_bytes"]
loglist_temp = ["duration_trans", "src_bytes_trans", "dst_bytes_trans"]
for i in range(len(loglist)):
    #spark_df = spark_df.withColumn(loglist_temp[i], log2(loglist[i]))
    spark_df = spark_df.withColumn(loglist_temp[i],when(col(loglist[i]) > 0, log(loglist[i])).otherwise(0))
    spark_df = spark_df.drop(loglist[i]).withColumnRenamed(loglist_temp[i], loglist[i])

#4 Chuyển đổi label data thành 0,1 
class_name = {"normal": "0", "anomaly": "1"}
spark_df = spark_df.replace(class_name, subset=["class"])
spark_df = spark_df.withColumn("class", col("class").cast(DoubleType()))
#class_df = spark_df.select("class")
#spark_df = spark_df.drop("class")
#5 Normalize Data
print("Sau khi scalling")
spark_df.show()
columntypes = spark_df.dtypes
for column, dtype in columntypes:
    sum_value = spark_df.agg(sum(col(column))).first()[0]
    if (sum_value == 0):
        print(column)
    elif (column == "class"):
        continue
    elif (column in translist):
        continue
    else:
        min_value = spark_df.select(min(col(column))).first()[0]
        max_value = spark_df.select(max(col(column))).first()[0]
        spark_df = spark_df.withColumn(column,(col(column) - min_value) / (max_value - min_value))
spark_df.show()

24/09/05 15:25:48 WARN TaskSetManager: Stage 145 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:49 WARN TaskSetManager: Stage 148 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:50 WARN TaskSetManager: Stage 151 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


Sau khi scalling


24/09/05 15:25:51 WARN TaskSetManager: Stage 154 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-----+-------------+-------+----+--------+------------------+-----------------+
|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_same_srv_rate|dst_host_diff_srv_rate|dst_

24/09/05 15:25:51 WARN TaskSetManager: Stage 155 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:51 WARN TaskSetManager: Stage 158 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:52 WARN TaskSetManager: Stage 161 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:53 WARN TaskSetManager: Stage 164 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:53 WARN TaskSetManager: Stage 167 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:54 WARN TaskSetManager: Stage 170 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:25:54 WARN TaskSetManager: Stage 173 contains a task of very large size (10740 KiB). The maximum recommended task size is 10

+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-------------+--------------+--------------------+--------------------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+-------------------+--------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-----+-------------+-------+----+--------+-------------------+-------------------+
|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|is_host_login|is_guest_login|               count|           srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|     dst_host_count|  

24/09/05 15:26:57 WARN TaskSetManager: Stage 500 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


In [ ]:
#Kiểm tra dữ liệu sau khi thực hiện tiền xử lý trước khi đưa vào học máy
#spark_df.show()
null_counts = {}
for checkcol in spark_df.columns:
    null_count = spark_df.filter(spark_df[checkcol].isNull()).count()
    null_counts[checkcol] = null_count
print(null_counts)
description_after = spark_df.describe()
description_after.show()

In [6]:
column_types1 = spark_df.dtypes
inputColumns = []
#outputCols = []
for column, dtype in columntypes:
    if (column == "class"):
        continue
    else:
        inputColumns.append(column)
print(inputColumns)
#print(outputCols)

assembler = VectorAssembler(inputCols=inputColumns, outputCol= "features")
#df_assembled = assembler.transform(spark_df).select("class", "features")
df_assembled = assembler.transform(spark_df)
df_final = df_assembled.select("features", "class")
df_final.show()
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=1234)
svm = LinearSVC(maxIter=10, regParam=0.1, labelCol="class", featuresCol="features")
cvModel = svm.fit(train_data)
predictions = cvModel.transform(test_data)
predictions.select("class", "features", "prediction").show()
evaluator = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Do chinh xac tren tap du lieu mau = {accuracy}")

['land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'protocol_type', 'service', 'flag', 'duration', 'src_bytes', 'dst_bytes']


24/09/05 15:27:11 WARN TaskSetManager: Stage 501 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|            features|class|
+--------------------+-----+
|(40,[15,16,21,24,...|  0.0|
|(40,[15,16,21,22,...|  0.0|
|(40,[15,16,17,18,...|  1.0|
|(40,[5,15,16,17,1...|  0.0|
|(40,[5,15,16,21,2...|  0.0|
|(40,[15,16,19,20,...|  1.0|
|(40,[15,16,17,18,...|  1.0|
|(40,[15,16,17,18,...|  1.0|
|(40,[15,16,17,18,...|  1.0|
|(40,[15,16,17,18,...|  1.0|
|(40,[15,16,19,20,...|  1.0|
|(40,[15,16,17,18,...|  1.0|
|(40,[5,15,16,21,2...|  0.0|
|(40,[5,15,16,21,2...|  1.0|
|(40,[15,16,17,18,...|  1.0|
|(40,[15,16,17,18,...|  1.0|
|(40,[5,15,16,18,2...|  0.0|
|(40,[15,16,21,24,...|  1.0|
|(40,[5,15,16,21,2...|  0.0|
|(40,[5,15,16,21,2...|  0.0|
+--------------------+-----+
only showing top 20 rows



24/09/05 15:27:12 WARN TaskSetManager: Stage 502 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:27:17 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/09/05 15:27:17 WARN TaskSetManager: Stage 503 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:27:18 WARN TaskSetManager: Stage 504 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:27:18 WARN TaskSetManager: Stage 505 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:27:18 WARN TaskSetManager: Stage 506 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:27:18 WARN TaskSetManager: Stage 507 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:27:18 WARN TaskSe

+-----+--------------------+----------+
|class|            features|prediction|
+-----+--------------------+----------+
|  0.0|(40,[0,15,16,17,1...|       1.0|
|  1.0|(40,[1,15,16,17,1...|       1.0|
|  1.0|(40,[1,15,16,17,2...|       1.0|
|  1.0|(40,[1,15,16,17,2...|       1.0|
|  1.0|(40,[1,15,16,17,2...|       1.0|
|  1.0|(40,[1,15,16,17,2...|       1.0|
|  1.0|(40,[1,15,16,17,2...|       1.0|
|  1.0|(40,[1,15,16,19,2...|       1.0|
|  1.0|(40,[1,15,16,19,2...|       1.0|
|  1.0|(40,[1,15,16,19,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
|  1.0|(40,[1,15,16,21,2...|       1.0|
+-----+--------------------+----------+
only showing top 20 rows



24/09/05 15:27:22 WARN TaskSetManager: Stage 529 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


Do chinh xac tren tap du lieu mau = 0.9576939851710876


In [7]:
#Training SVM model
#Gộp các cột thành 1 vector cột
column_types1 = spark_df.dtypes
inputColumns = []
#outputCols = []
for column, dtype in columntypes:
    if (column == "class"):
        continue
    else:
        inputColumns.append(column)
print(inputColumns)
#print(outputCols)

assembler = VectorAssembler(inputCols=inputColumns, outputCol= "features")
#df_assembled = assembler.transform(spark_df).select("class", "features")
df_assembled = assembler.transform(spark_df)
selector = ChiSqSelector(numTopFeatures=10, featuresCol="features", outputCol="selected_features", labelCol="class")
selectorModel = selector.fit(df_assembled)
df_selected = selectorModel.transform(df_assembled)
# Prepare DataFrame for model training
df_final = df_selected.select("selected_features", "class")
df_final.show()

train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=1234)

#svm = LinearSVC(maxIter=10, regParam=0.1, labelCol="class", featuresCol="selected_features")
#cvModel = svm.fit(train_data)
#predictions = cvModel.transform(test_data)
svm = LinearSVC(labelCol="class", featuresCol="selected_features")
paramGrid = ParamGridBuilder() \
    .addGrid(svm.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(svm.maxIter, [10, 50, 100]) \
    .build()
evaluator = BinaryClassificationEvaluator(labelCol="class", rawPredictionCol="rawPrediction")
crossval = CrossValidator(estimator=svm,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)
cvModel = crossval.fit(train_data)
#cvModel = crossval.fit(df_assembled)
predictions = cvModel.transform(test_data)

#predictions.select("class", "features", "prediction").show()
predictions.select("class", "selected_features", "prediction").show()
evaluator = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Do chinh xac tren tap du lieu mau = {accuracy}")


['land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'protocol_type', 'service', 'flag', 'duration', 'src_bytes', 'dst_bytes']


24/09/05 15:27:59 WARN TaskSetManager: Stage 531 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:27:59 WARN TaskSetManager: Stage 532 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:28:00 WARN TaskSetManager: Stage 534 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:28:02 WARN TaskSetManager: Stage 536 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|   selected_features|class|
+--------------------+-----+
|(10,[8,9],[0.0039...|  0.0|
|(10,[8,9],[0.0254...|  0.0|
|(10,[8,9],[0.2407...|  1.0|
|(10,[2,8,9],[1.0,...|  0.0|
|(10,[2,8,9],[1.0,...|  0.0|
|(10,[8,9],[0.2367...|  1.0|
|(10,[8,9],[0.3248...|  1.0|
|(10,[8,9],[0.2289...|  1.0|
|(10,[8,9],[0.5283...|  1.0|
|(10,[8,9],[0.2602...|  1.0|
|(10,[8,9],[0.4011...|  1.0|
|(10,[8,9],[0.3894...|  1.0|
|(10,[2,8,9],[1.0,...|  0.0|
|(10,[2,8,9],[1.0,...|  1.0|
|(10,[8,9],[0.4559...|  1.0|
|(10,[8,9],[0.1878...|  1.0|
|(10,[2,8,9],[1.0,...|  0.0|
|(10,[8,9],[0.0019...|  1.0|
|(10,[2,8,9],[1.0,...|  0.0|
|(10,[2,8,9],[1.0,...|  0.0|
+--------------------+-----+
only showing top 20 rows



24/09/05 15:28:03 WARN TaskSetManager: Stage 537 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:28:04 WARN TaskSetManager: Stage 538 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:28:05 WARN TaskSetManager: Stage 539 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:28:05 WARN TaskSetManager: Stage 540 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:28:05 WARN TaskSetManager: Stage 541 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:28:05 WARN TaskSetManager: Stage 542 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 15:28:05 WARN TaskSetManager: Stage 543 contains a task of very large size (10740 KiB). The maximum recommended task size is 10

+-----+--------------------+----------+
|class|   selected_features|prediction|
+-----+--------------------+----------+
|  1.0|(10,[0,8,9],[0.33...|       1.0|
|  1.0|(10,[0,8,9],[0.33...|       1.0|
|  1.0|(10,[0,8,9],[0.33...|       1.0|
|  1.0|(10,[0,8,9],[0.33...|       1.0|
|  1.0|(10,[0,8,9],[0.33...|       1.0|
|  1.0|(10,[0,8,9],[0.33...|       1.0|
|  1.0|(10,[0,8,9],[0.33...|       1.0|
|  1.0|(10,[0,8,9],[0.33...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
|  1.0|(10,[0,8,9],[1.0,...|       1.0|
+-----+--------------------+----------+
only showing top 20 rows



24/09/05 15:41:43 WARN TaskSetManager: Stage 9089 contains a task of very large size (10740 KiB). The maximum recommended task size is 1000 KiB.


Do chinh xac tren tap du lieu mau = 0.8324808691170057


In [8]:
#Sau khi training model xong, thuc hien tren data tu file Test
file_path= '/root/NSLKDDProject/nslkdd/KDDTest+.arff'
df_test = ldata.load_arff_to_pandasDF(file_path)
#Load DataFrame
spark_df_test = spark.createDataFrame(df_test)
#Show dữ liệu 
spark_df_test.show()
class_counts = spark_df_test.groupBy("class").count()
class_counts.show()

24/09/05 17:05:30 WARN TaskSetManager: Stage 9091 contains a task of very large size (1739 KiB). The maximum recommended task size is 1000 KiB.
24/09/05 17:05:30 WARN TaskSetManager: Stage 9092 contains a task of very large size (1739 KiB). The maximum recommended task size is 1000 KiB.


+--------+-------------+--------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type| service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_h

In [ ]:
# THỰC HIỆN PREPROCESSING DATA BAO GỒM
# Drop cột
spark_df_test = spark_df_test.drop("num_outbound_cmds")
#1. One hot encoding data string dựa trên tần suất xuất hiện của value trong data
translist = ["protocol_type", "service", "flag"]
translist_temp = ["protocol_type_trans", "service_trans", "flag_trans"]
for i in range(len(translist)):
    indexer1 = StringIndexer(inputCol=translist[i], outputCol=translist_temp[i])
    spark_df_test = indexer1.fit(spark_df_test).transform(spark_df_test)
    spark_df_test = spark_df_test.drop(translist[i]).withColumnRenamed(translist_temp[i], translist[i])

#2. Chuyển đổi các cột string cast về double
castlistvalue = ["land", "logged_in", "is_host_login", "is_guest_login"]
for i in range (len(castlistvalue)):
    spark_df_test = spark_df_test.withColumn(castlistvalue[i], col(castlistvalue[i]).cast(DoubleType()))
#3 Tách label data ra Dataframe mới
class_name = {"normal": "0", "anomaly": "1"}
spark_df_test = spark_df_test.replace(class_name, subset=["class"])
spark_df_test = spark_df_test.withColumn("class", col("class").cast(DoubleType()))

loglist = ["duration", "src_bytes", "dst_bytes"]
loglist_temp = ["duration_trans", "src_bytes_trans", "dst_bytes_trans"]
for i in range(len(loglist)):
    #spark_df = spark_df.withColumn(loglist_temp[i], log2(loglist[i]))
    spark_df_test = spark_df_test.withColumn(loglist_temp[i],when(col(loglist[i]) > 0, log(loglist[i])).otherwise(0))
    spark_df_test = spark_df_test.drop(loglist[i]).withColumnRenamed(loglist_temp[i], loglist[i])
#class_df = spark_df.select("class")
#spark_df = spark_df.drop("class")
#4 Normalize Data
columntypes = spark_df_test.dtypes
for column, dtype in columntypes:
    sum_value = spark_df_test.agg(sum(col(column))).first()[0]
    if (sum_value == 0):
        print(column)
    elif (column == "class"):
        continue
    else:
        min_value = spark_df_test.select(min(col(column))).first()[0]
        max_value = spark_df_test.select(max(col(column))).first()[0]
        spark_df_test = spark_df_test.withColumn(column,(col(column) - min_value) / (max_value - min_value))
spark_df_test.show()


In [ ]:
des = spark_df_test.describe()
des.show()
column_types1 = spark_df_test.dtypes
inputColumns = []
#outputCols = []
for column, dtype in columntypes:
    if (column == "class"):
        continue
    else:
        inputColumns.append(column)
print(inputColumns)
#print(outputCols)

#df_test_assembled = assembler.transform(spark_df_test).select("class", "features")
df_test_assembled = assembler.transform(spark_df_test)
df_test_selected = selectorModel.transform(df_test_assembled)

# Prepare DataFrame for making predictions
df_test_final = df_test_selected.select("selected_features", "class")

predictions = cvModel.transform(df_test_final)
#predictions = cvModel.transform(df_test_assembled)
#predictions.select("class", "features", "prediction").show()
evaluator = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Do chinh xac tren tap du lieu test = {accuracy}")
truep = predictions.filter((col("class") == 1) & (col("prediction") == 1)).count()
truen = predictions.filter((col("class") == 0) & (col("prediction") == 0)).count()
falsep = predictions.filter((col("class") == 0) & (col("prediction") == 1)).count()
falsen = predictions.filter((col("class") == 1) & (col("prediction") == 0)).count()
print(f"True Positives: {truep}")
print(f"True Negatives: {truen}")
print(f"False Positives: {falsep}")
print(f"False Negatives: {falsen}")